In [ ]:
import pydrake

设置求解器的参数有两种方法: 调用 `MathematicalProgram::SetSolverOption` 方法, 或者给 `Solve()` 方法传参

# 调用 `MathematicalProgram::SetSolverOption` 方法

通过调用 `MathematicalProgram::SetSolverOption(solver_id, option_name, option_value)`, 可以为特定的 `solver_id` 求解器设置参数, `option_name` 和求解器有关, 例如 IPOPT 参数列表. `MathematicalProgram` 对象存储该求解器参数, 调用求解器是, 则此参数将应用于 `Solve()` 方法的调用中. 下面的例子展示了设置 IPOPT 参数

In [ ]:
from pydrake.solvers import MathematicalProgram, Solve, SolverOptions, IpoptSolver
import numpy as np

prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddCost(x[0]**2 + x[1] ** 2)
prog.AddConstraint(x[0] + x[1] == 1)

# Set the maximum iteration for IPOPT to be 1.
# max_iter is a parameter of IPOPT solver, explained in
# https://coin-or.github.io/Ipopt/OPTIONS.html
prog.SetSolverOption(IpoptSolver().solver_id(), "max_iter", 1)
solver = IpoptSolver()
result = solver.Solve(prog, np.array([10, 1]), None)
# With fewer maximum iteration, IPOPT hasn't converged to optimality yet (The true optimal is [0.5, 0.5])
print("Success? ", result.is_success())
print(result.get_solution_result())
print("IPOPT x*= ", result.GetSolution(x))

值得注意的是, 设置求解器的参数并不意味着 deake 会调用该求解器. 和之前说的一样, Drake决定调用其认为最合适的求解器. 比如, 尽管我们为 IPOPT 设置了参数, Drake 选择了另一个解算器(它可以求这个特定问题的解析解). 

In [ ]:
prog.SetSolverOption(IpoptSolver().solver_id(), "max_iter", 1)
result = Solve(prog)
print(result.get_solver_id().name())

# 通过传参的方式

In [ ]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddCost(x[0]**2 + x[1] ** 2)
prog.AddConstraint(x[0] + x[1] == 1)

solver_options = SolverOptions()
solver_options.SetOption(IpoptSolver().solver_id(), "max_iter", 1)
solver = IpoptSolver()

# Call Solve with solver_options, IPOPT will use `max_iter` = 1
result = solver.Solve(prog, np.array([10, 1]), solver_options)
print("Success? ", result.is_success())
print(result.get_solution_result())
# Call Solve without solver_options, IPOPT will use the default options.
result = solver.Solve(prog, np.array([10, 1]), None)
print("Success? ", result.is_success())
print(result.get_solution_result())